In [1]:
from lm_polygraph.utils import UEManager

man_test: UEManager = UEManager.load('llama8b_trivia.man')
man_cal: UEManager = UEManager.load('llama_triviaqa_train_sample_no_embeddings.man')


/home/nkotelevskii/github/multidimensional_uncertainty/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nkotelevskii/github/multidimensional_uncertainty/.venv/lib/python3.12/site-packages/lm_polygraph/utils/manager.py:705: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless the

Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f6b5b0d6660>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f6ae530e870>]


In [2]:
man_cal.estimations

defaultdict(list, {})

In [3]:
for el in man_cal.estimations.keys():
    print(el)

In [4]:
for el in man_test.estimations.keys():
    print(el)


('sequence', 'GreedySemanticEnrichedPPLAveDissimilarity')
('sequence', 'GreedySemanticEnrichedMaxprobAveDissimilarity')
('sequence', 'GreedySemanticEnrichedMTEAveDissimilarity')
('sequence', 'BestSemanticEnrichedPPLAveDissimilarity')
('sequence', 'BestSemanticEnrichedMaxprobAveDissimilarity')
('sequence', 'BestSemanticEnrichedMTEAveDissimilarity')
('sequence', 'MbrSemanticEnrichedPPLAveDissimilarity')
('sequence', 'MbrSemanticEnrichedMaxprobAveDissimilarity')
('sequence', 'MbrSemanticEnrichedMTEAveDissimilarity')
('sequence', 'SemanticEntropy')
('sequence', 'SAR_t0.001')
('sequence', 'MaximumSequenceProbability')
('sequence', 'Perplexity')
('sequence', 'MeanTokenEntropy')
('sequence', 'BestSampledMaximumSequenceProbability')
('sequence', 'BestSampledPerplexity')
('sequence', 'BestSampledMeanTokenEntropy')
('sequence', 'MbrSampledMaximumSequenceProbability')
('sequence', 'MbrSampledPerplexity')
('sequence', 'MbrSampledMeanTokenEntropy')
('sequence', 'MonteCarloSequenceEntropy')
('sequen

In [11]:
from mdu.unc.entropic_ot import EntropicOTOrdering
from mdu.unc.constants import OTTarget, SamplingMethod, ScalingType
import numpy as np

In [ ]:
final_measures = {}

hyperparams_list = [
    {
        'target': OTTarget.EXP,
        'sampling_method': SamplingMethod.GRID,
        'scaling_type': ScalingType.FEATURE_WISE,
        'grid_size': 5,
        'n_targets_multiplier': 1,
        'eps': 0.5,
        'max_iters': 1000,
        'tol': 1e-6,
    },
    {
        'target': OTTarget.EXP,
        'sampling_method': SamplingMethod.GRID,
        'scaling_type': ScalingType.GLOBAL,
        'grid_size': 5,
        'n_targets_multiplier': 1,
        'eps': 0.5,
        'max_iters': 1000,
        'tol': 1e-6,
    },
    {
        'target': OTTarget.BETA,
        'sampling_method': SamplingMethod.GRID,
        'scaling_type': ScalingType.FEATURE_WISE,
        'grid_size': 5,
        'n_targets_multiplier': 1,
        'eps': 0.5,
        'max_iters': 1000,
        'tol': 1e-6,
    },
    {
        'target': OTTarget.BETA,
        'sampling_method': SamplingMethod.GRID,
        'scaling_type': ScalingType.GLOBAL,
        'grid_size': 5,
        'n_targets_multiplier': 1,
        'eps': 0.5,
        'max_iters': 1000,
        'tol': 1e-6,
    },
]
for hyperparams in hyperparams_list:
    target = hyperparams['target']
    sampling_method = hyperparams['sampling_method']
    scaling_type = hyperparams['scaling_type']
    grid_size = hyperparams['grid_size']
    n_targets_multiplier = hyperparams['n_targets_multiplier']
    eps = hyperparams['eps']
    max_iters = hyperparams['max_iters']
    tol = hyperparams['tol']


    model = EntropicOTOrdering(
        target=target,
        sampling_method=sampling_method,
        scaling_type=scaling_type,
        grid_size=grid_size,
        target_params={},
        eps=eps,
        n_targets_multiplier=n_targets_multiplier,
        max_iters=max_iters,
        random_state=42,
        tol=tol,
    )

    combinations = [
        [f"only_confidence_{target.value}_{scaling_type.value}", (('sequence', 'MaximumSequenceProbability'), ('sequence', 'Perplexity'), ('sequence', 'MeanTokenEntropy'), ('sequence', 'MonteCarloSequenceEntropy'), ('sequence', 'MonteCarloNormalizedSequenceEntropy'))],
        [f"only_consistency_1_{target.value}_{scaling_type.value}", (('sequence', 'EigValLaplacian_NLI_score_entail'), ('sequence', 'DegMat_NLI_score_entail'))],
        [f"only_consistency_2_{target.value}_{scaling_type.value}", (('sequence', 'GreedySemanticEnrichedPPLAveDissimilarity'), ('sequence', 'GreedySemanticEnrichedMaxprobAveDissimilarity'), ('sequence', 'GreedySemanticEnrichedMTEAveDissimilarity'), ('sequence', 'BestSemanticEnrichedPPLAveDissimilarity'), ('sequence', 'BestSemanticEnrichedMaxprobAveDissimilarity'), ('sequence', 'BestSemanticEnrichedMTEAveDissimilarity'), ('sequence', 'MbrSemanticEnrichedPPLAveDissimilarity'), ('sequence', 'MbrSemanticEnrichedMaxprobAveDissimilarity'), ('sequence', 'MbrSemanticEnrichedMTEAveDissimilarity'))],
        [f"comb_1_{target.value}_{scaling_type.value}", (('sequence', 'GreedyAveDissimilarity'), ('sequence', 'MaximumSequenceProbability'))],
        [f"comb_2_{target.value}_{scaling_type.value}", (('sequence', 'GreedyAveDissimilarity'), ('sequence', 'Perplexity'))],
        [f"comb_3_{target.value}_{scaling_type.value}", (('sequence', 'GreedyAveDissimilarity'), ('sequence', 'MeanTokenEntropy'))],
        [f"comb_4_{target.value}_{scaling_type.value}", (('sequence', 'GreedyAveDissimilarity'), ('sequence', 'Perplexity'), ('sequence', 'MeanTokenEntropy'), ('sequence', 'MaximumSequenceProbability'))],
    
    ]

    for comb_list in combinations:
        name, comb = comb_list[0], comb_list[1]
        components = [man_test.estimations[el] for el in comb]
        stacked_components = np.column_stack(components)
        
        calib_components = [man_cal.estimations[el] for el in comb]

        stacked_calib_components = np.column_stack(calib_components)
        stacked_test_components = stacked_components

        model.fit(stacked_calib_components)
        scores = model.predict(stacked_test_components)

        final_measures[name] = scores

In [6]:
import pickle

# Save the final_measures dictionary
with open('final_measures.pkl', 'wb') as f:
    pickle.dump(final_measures, f)

print("final_measures saved to 'final_measures.pkl'")

final_measures saved to 'final_measures.pkl'


In [ ]:
import pickle

with open('final_measures.pkl', 'rb') as f:
    final_measures_1 = pickle.load(f)

print("final_measures_1 loaded from 'final_measures_1.pkl'")
print(f"Keys in final_measures_1: {list(final_measures_1.keys())}")


final_measures_1 loaded from 'final_measures_1.pkl'
Keys in final_measures_1: ['only_confidence_Exp_FeatureWise', 'only_consistency_1_Exp_FeatureWise', 'only_consistency_2_Exp_FeatureWise', 'comb_1_Exp_FeatureWise', 'comb_2_Exp_FeatureWise', 'comb_3_Exp_FeatureWise', 'comb_4_Exp_FeatureWise', 'only_confidence_Exp_Global', 'only_consistency_1_Exp_Global', 'only_consistency_2_Exp_Global', 'comb_1_Exp_Global', 'comb_2_Exp_Global', 'comb_3_Exp_Global', 'comb_4_Exp_Global', 'only_confidence_Beta_FeatureWise', 'only_consistency_1_Beta_FeatureWise', 'only_consistency_2_Beta_FeatureWise', 'comb_1_Beta_FeatureWise', 'comb_2_Beta_FeatureWise', 'comb_3_Beta_FeatureWise', 'comb_4_Beta_FeatureWise', 'only_confidence_Beta_Global', 'only_consistency_1_Beta_Global', 'only_consistency_2_Beta_Global', 'comb_1_Beta_Global', 'comb_2_Beta_Global', 'comb_3_Beta_Global', 'comb_4_Beta_Global']


In [8]:
final_measures_1

{'only_confidence_Exp_FeatureWise': array([0.7146543 , 0.71251383, 0.9671379 , ..., 1.12020473, 1.07814127,
        1.12374437]),
 'only_consistency_1_Exp_FeatureWise': array([0.39026273, 0.40058319, 0.46072534, ..., 1.09118368, 0.85247064,
        1.23412204]),
 'only_consistency_2_Exp_FeatureWise': array([0.90893293, 0.90920409, 0.96788134, ..., 1.10687898, 1.07107922,
        1.08856114]),
 'comb_1_Exp_FeatureWise': array([0.44390064, 0.44452144, 0.5885862 , ..., 0.84368582, 0.81117989,
        0.89605164]),
 'comb_2_Exp_FeatureWise': array([0.41845704, 0.42117843, 0.6106255 , ..., 0.86482238, 0.88043151,
        0.96881132]),
 'comb_3_Exp_FeatureWise': array([0.39188181, 0.40101038, 0.68577445, ..., 0.9069123 , 0.98211785,
        1.01031097]),
 'comb_4_Exp_FeatureWise': array([0.63383479, 0.64000304, 0.85277894, ..., 1.02307114, 1.03487073,
        1.06609061]),
 'only_confidence_Exp_Global': array([0.85073486, 0.84829506, 0.91636073, ..., 0.99632216, 0.94002092,
        0.9700823